# Load Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import time
import json
import csv

import pandas as pd
import numpy as np

from tqdm import tqdm
from collections import Counter
from pydub import AudioSegment # sudo apt install ffmpeg
from pprint import pprint

from functions.functions_cough import get_cough, convert_events_to_seconds, label_generator, slice_audio, audiosegment_to_amplitudes

# Main

In [ ]:
list_dataset_name = [
    'coswara', 
    'coughvid', 
    'esc50', 
    'fsdkaggle', 
    'virufy',
    ]

new_sample_rate = 16000  # New sample rate in Hz

if not os.path.exists(f'Results/Sliced_Wav'):
    os.makedirs(f'Results/Sliced_Wav')

In [ ]:
for window_size in [1, 5, 10]:
    for dataset_name in list_dataset_name:
        print(dataset_name, window_size)
        
        file_path_save = f"Results/Sliced_Wav/{dataset_name}_{window_size}/"
        
        if not os.path.exists(file_path_save):
            os.makedirs(file_path_save)
        
        df_all = pd.read_csv(f'Results/Data/data_summary_{dataset_name}.csv')
        df_all = df_all.sample(frac=1).groupby('label').head(1000).reset_index(drop=True)
            
        df_results = []
            
        for i in tqdm(range(len(df_all))):
            
            try:
            # if True:
                data_set = df_all['dataset'][i]
                file_path = df_all['filepath'][i]
                file_name = df_all['filename'][i]
    
                age = df_all['age'][i]
                gender = df_all['gender'][i]
                label = df_all['label'][i]
                status = df_all['status'][i]
    
                # Get slice of audio
                slices = slice_audio(file_path, window_size, new_sample_rate)
    
                for j, slice in enumerate(slices):
                    file_name_save = f"{file_name}_{j}.wav"
                    mean = np.mean(np.abs(audiosegment_to_amplitudes(slice)))
                    slice.export(file_path_save + file_name_save, format="wav")
    
                    results = [data_set, 
                               file_path, file_name, 
                               file_path_save + file_name_save, file_name_save,
                               age, gender, label, status,  mean]
    
                    df_results.append(results)
                    
            except Exception as error:
                pass
                      
        columns = ['dataset', 
                   'filepath', 'filename',
                   'filepathslice', 'filenameslice',
                   'age', 'gender', 'label', 'status', 'mean_amplitude']
        
        df_results = pd.DataFrame(df_results, columns=columns)
        df_results.to_csv(f'Results/Sliced_Wav/dataset_{data_set}_{window_size}s.csv', index=False)


In [ ]:
df_results

In [ ]:
audiosegment_to_amplitudes(slice)